# Shor's Algorithm
This implementation was authored by Gregory Croisdale <br>
5/1/2020, COSC 494, University of Tennesse, Knoxville

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def collapse(n, a, rem=-1):
    """Find states which yield certain remainder"""
    # initialize remainder and catch stop recursion case
    if rem == -1:
        rem = n - 1
    if rem == 0:
        raise ValueError("No valid states found")
    
    # find mod inverses
    end = 2 ** n.bit_length()
    out = []
    for i in range(end):
        if (a ** i) % n == rem:
            out += [i]
    
    # if unsuccessful, try reducing remainder
    if len(out) == 0:
        return collapse(n, a, rem=rem-1)
    
    return out

In [3]:
def quantum_ft(n, guess):
    """Performs a quantum fourier transformation so we can find periods. Large
       portions of this code are from the Qiskit reference material.           """
    from qiskit import QuantumCircuit, QuantumRegister, execute, Aer, IBMQ
    from numpy import pi
    
    # The following code is from qiskit reference:
    def qft_rotations(circuit, n):
        """Performs qft on the first n qubits in circuit (without swaps)"""
        if n == 0:
            return circuit
        n -= 1
        circuit.h(n)
        for qubit in range(n):
            circuit.cu1(pi/2**(n-qubit), qubit, n)
        # At the end of our function, we call the same function again on
        # the next qubits (we reduced n by one earlier in the function)
        qft_rotations(circuit, n)
    
    def swap_registers(circuit, n):
        for qubit in range(n//2):
            circuit.swap(qubit, n-qubit-1)
        return circuit

    def qft(circuit, n):
        """QFT on the first n qubits in circuit"""
        qft_rotations(circuit, n)
        swap_registers(circuit, n)
        return circuit
    
    # The following code is my own:
    # mark qubits with designated modulo value
    marked = collapse(n, guess)
    
    # create a democratic state from all marked qubits
    state = [1 / np.sqrt(len(marked)) if i in marked else 0 for i in range(2 ** n.bit_length())]
    q = QuantumRegister(n.bit_length())
    qc = QuantumCircuit(q)
    qc.initialize(state, q)
    
    # perform qft twice
    qft(qc,n.bit_length())
    qft(qc,n.bit_length())

    backend = Aer.get_backend("statevector_simulator")
    statevector = execute(qc, backend=backend).result().get_statevector()
    return np.abs(statevector) ** 2

In [4]:
def classical_ft(n):
    """Performs a classical fourier transformation so we can find periods."""
    return []

In [12]:
def find_period(ft_result):
    """Finds the period from a completed fourier transformation."""
    probs = np.abs((ft_result)) ** 2
    peaks = np.where(probs > np.amax(probs) / 2)[0]
    if len(peaks) < 2:
        return 1
    return peaks[1] - peaks[0]

In [6]:
def prime_from_period(x, r, n):
    """Turns a found period into two factors"""
    prime1 = np.gcd(x ** (r // 2) - 1, n)
    if prime1 == 1:
        prime1 = np.gcd(x ** (r // 2) + 1, n)
    prime2 = n // prime1
    assert prime1 * prime2 == n
    
    return prime1, prime2

In [7]:
def shor_factor(
    n,
    guess=3,
    mode="quantum"):
    """Factors an integer using a designated method"""
    if (mode == "quantum"):
        while (guess < n):
            print(guess)
            period = find_period(quantum_ft(n, guess))
            prime1, prime2 = prime_from_period(guess, period, n)
            
            if (abs(prime1 - prime2) != n - 1):
                return prime1, prime2
            
            guess += 1
            while np.gcd(guess, n) != 1 and guess < n:
                guess += 1
                
    elif (mode == "classical_ft"):
        return find_period(quantum_ft(n, randint(2, n - 1)))
    
    elif ()
    
    else:
        raise TypeError(mode)
        
    return 1, n

In [17]:
import matplotlib.pyplot as plt

p1, p2 = shor_factor(199, mode="quantum")
print(p1, p2)

3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
1 199
